# Preparation des nos variables de sortie

Dans cette partie nous allons preparer notre ensemble de sortie en construisant nos varibles de sortie , celle ci seront 
constiruer à l'aide  des divers fonction qui nous aiderons à determiner pour une année si sur base des résultat que nous aons si un étudiant est passé en premiere seion, s'il a distingué ou bien s'il a échoue, nous determinerons enfin  le pourcentage obtebnu.

Notons que nous nous sommes butés à 2 grands problème concernant les données obtenus et qui risquent de biaiser nos calculs:
    - Nous n'avons pas pu obtenir le pourcentage final de certains étudiant ayant echouer ou abandonner leurs études avant la fin de l'année académique ces étudiants qui après la seconde session sont AA (Assimilé aux Ajournées ) n'ont pas de pourcentage final pour palier àce probleme nous avons remplacer cette valeur manquante par une valeur aleatoiree choisie entre 40 et 50 pourcent ce qui represente l'echec!
    - Pour nous étudiant le nombre des années académiques pour lequel nous avons le résultat n'est pas le meme  pour certains étudiant nous avons des informations pour 1 annéé académique, d'autres pour 2 , etc. Pour palier à ce problème nous avons utliser des ration pour nous valeur de sortie qui consiter à diviser le nombre sur le no,bre des année académique que l'étudiant a passée à la faculté


Comencons par charger nos libraries 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns #for beatiful visualizations
%matplotlib inline 
import scipy.stats as scs #for statistics
import operator
from scipy.stats import chi2_contingency
import matplotlib.ticker as ticker

Notons que dans cette partie nous allons Aussi faire une analyse statistique univarié et bi-varié entre nos données d"entree et celle de sortie

In [2]:
dataset=pd.read_csv("DatasetOut.csv",index_col="Unnamed: 0")

In [3]:
dataset.rename(columns={'IDENTIFICATION':'ID'},inplace=True)

In [4]:
SuccefullMent=['S']
PartialSuccess=['ADM', 'ADTFC', 'ADSTM', 'ADSTFC', 'ADST']
Distinction=['D','GD']
FailSession=['AA','AR','nan']
FailureMention=['A','NAF','ANAF']

In [5]:
dataset.MENT1.value_counts()

AA        4569
A         2351
S         2037
D          268
ADM        101
ADTFC       81
ADSTM       74
ADSTFC      55
ADST        45
NAF         17
GD           7
ANAF         1
Name: MENT1, dtype: int64

In [6]:
dataset.columns

Index([u'ID', u'ACADYEAR', u'PERC1', u'MENT1', u'PERC2', u'MENT2', u'FAC',
       u'OPT', u'PROM'],
      dtype='object')

Premierement Voici la fonction que nous avons utiliser pour determiner si un étudiant a reussi ou a echouer pour une année academique

In [8]:
def calculateSucess(x):
    """this function will check if a student pass of fail in an aacademic year"""
    if str(x['MENT2']) in FailSession+FailureMention and (str(x['MENT1']) not in SuccefullMent+PartialSuccess+Distinction):
        return pd.Series(True)
    else :
        return pd.Series(False)

In [10]:
Echec=dataset.apply(calculateSucess,axis=1)

In [12]:
Echec[:6]

,0
0,True
1,False
2,True
3,False
4,False
5,True


In [22]:
dataset[:6]

,ID,ACADYEAR,PERC1,MENT1,PERC2,MENT2,FAC,OPT,PROM,Echec
0,45,2013-2014,NaN,AA,NaN,NaN,FPSE,Sciences de l'Éducation,L2,True
1,215,2012-2013,NaN,ADM,63.099998,S,FD,Droit Privé et Judiciaire,L2,False
2,343,2015-2016,NaN,AA,52.200001,A,FSEG,Tronc commun,G2,True
3,356,2015-2016,NaN,ADSTM,59.900002,S,FSEG,Gestion des Entreprises,L2,False
4,398,2012-2013,65.099998,S,NaN,NaN,FSDC,Santé et Développement Communautaires,L2,False
5,429,2013-2014,NaN,AA,NaN,A,FD,Tronc commun,G1,True


Avec cette unction nous avons pu determier si un étudiant a reussi ou echoué dans une année académique

In [21]:
dataset['Echec']=Echec

Ensuite On cherche à déterminer si l'étudiant est passé en premiere session dans une anné academique 

Voici la fonction qui nous permet d'effectuer ses calculs:

In [27]:
def calculatePass1estSes(x):
    """this function will check if a student pass in the 1st session in an academic year"""
    if x['MENT1'] in SuccefullMent+Distinction:
        return pd.Series(True)
    else :
        return pd.Series(False)

In [28]:
Pass1erSession=dataset.apply(calculatePass1estSes,axis=1)

In [29]:
dataset.loc[:,'Pass1erSession']=Pass1erSession
dataset['Pass1erSession']=Pass1erSession

In [30]:
dataset.head(8)

,ID,ACADYEAR,PERC1,MENT1,PERC2,MENT2,FAC,OPT,PROM,Echec,Pass1erSession
0,45,2013-2014,NaN,AA,NaN,NaN,FPSE,Sciences de l'Éducation,L2,True,False
1,215,2012-2013,NaN,ADM,63.099998,S,FD,Droit Privé et Judiciaire,L2,False,False
2,343,2015-2016,NaN,AA,52.200001,A,FSEG,Tronc commun,G2,True,False
3,356,2015-2016,NaN,ADSTM,59.900002,S,FSEG,Gestion des Entreprises,L2,False,False
4,398,2012-2013,65.099998,S,NaN,NaN,FSDC,Santé et Développement Communautaires,L2,False,True
5,429,2013-2014,NaN,AA,NaN,A,FD,Tronc commun,G1,True,False
6,474,2014-2015,NaN,AA,NaN,NaN,FD,Droit Privé et Judiciaire,G3,True,False
7,474,2015-2016,62.500000,S,NaN,NaN,FD,Droit Privé et Judiciaire,G3,False,True


Ensuite on cherche à determiner si un étudiant a distinguer au cours d'une académique , voicila function:

In [34]:
def calculateDistinCtion(x):
    """this function will check if a student get a distinction mention in aacademic year"""
    if x['MENT1'] in Distinction or x['MENT2']in Distinction:
        return pd.Series(True)
    else :
        return pd.Series(False)

In [35]:
Distinction=dataset.apply(calculateDistinCtion,axis=1)

In [36]:
dataset.loc[:,'Distinction']=Distinction
dataset['Distinction']=Distinction

In [39]:
dataset.loc[dataset.Distinction].head(8)

,ID,ACADYEAR,PERC1,MENT1,PERC2,MENT2,FAC,PROM,Echec,Pass1erSession,Distinction
47,2221,2012-2013,70.199997,D,NaN,NaN,FD,L2,False,True,True
57,2457,2012-2013,76.400002,D,NaN,NaN,FSTA,TECH2,False,True,True
96,3298,2015-2016,70.699997,D,NaN,NaN,FPSE,L2,False,True,True
106,3578,2012-2013,70.099998,D,NaN,NaN,FD,L2,False,True,True
110,3588,2013-2014,NaN,AA,73.500000,D,FSTA,Tech1,False,False,True
114,3591,2013-2014,NaN,ADSTM,73.900002,D,FSTA,Tech2,False,False,True
123,3626,2014-2015,71.900002,D,NaN,NaN,FSTA,Tech1,False,True,True
140,3646,2013-2014,NaN,AA,73.300003,D,FSTA,Tech1,False,False,True


Enfin Calculons le pourcentage final qu'a obtenu un étudiant au cours d'une anné academique 

In [155]:
def calculatePourcenatge(x):
    """this function will check if a student pass of fail in an aacademic year"""
    if x['Pass1erSession'] :
        return pd.Series(x['PERC1'])
    elif x['MENT2'] in SuccefullMent and  x['MENT1'] in FailSession+FailureMention+PartialSuccess:
        return pd.Series(x['PERC2'])
    elif str(x['MENT2']) in Distinction and  str(x['MENT1']) in FailSession+FailureMention+PartialSuccess :
        return pd.Series(x['PERC2'])
    else:
        return pd.Series(x['PERC2'])

In [116]:
def calculateDistinCtion2emeS(x):
    if str(x['MENT2']) in Distinction :
        return pd.Series(x['MENT2'])

In [156]:
PercFinal=dataset.apply(calculatePourcenatge,axis=1)

In [157]:
PercFinal.loc[dataset.Pass1erSession]

,0
4,65.099998
7,62.500000
8,65.000000
9,60.400002
10,68.000000
17,62.900002
18,68.800003
23,62.400002
28,66.500000
34,65.099998


In [168]:
dataset.loc[:,'PercFinal']=PercFinal
dataset['PercFinal']=PercFinal

il ya des étudiant qui on reussi mais qui n'ont  pas de pourcentage final

In [169]:
dataset.loc[operator.and_(np.isnan(PercFinal.get(0)) ,operator.or_(dataset.MENT2=='S',dataset.MENT1=='S'))]

,ID,ACADYEAR,PERC1,MENT1,PERC2,MENT2,FAC,PROM,Echec,Pass1erSession,Distinction,PercFinal
248,3978,2013-2014,NaN,AA,NaN,S,FSDC,L2,False,False,False,NaN
2176,7150,2012-2013,50.400002,A,NaN,S,FSEG,G2,False,False,False,NaN
2700,7530,2012-2013,NaN,S,NaN,NaN,FSTA,G1,False,True,False,NaN
2739,7569,2012-2013,NaN,S,NaN,NaN,FSTA,G1,False,True,False,NaN
2837,7630,2012-2013,NaN,S,NaN,NaN,FSTA,G1,False,True,False,NaN
3022,7750,2013-2014,NaN,ADSTFC,NaN,S,FD,G3,False,False,False,NaN
3100,7804,2013-2014,NaN,AA,NaN,S,FD,G3,False,False,False,NaN
3701,8300,2012-2013,NaN,S,NaN,NaN,FSTA,G0,False,True,False,NaN
4089,8477,2012-2013,NaN,S,NaN,NaN,FSTA,G0,False,True,False,NaN
6648,9688,2012-2013,NaN,S,NaN,NaN,FSTA,G0,False,True,False,NaN


In [170]:
dataset.loc[operator.and_(np.isnan(PercFinal.get(0)) ,operator.or_(dataset.MENT2=='S',dataset.MENT1=='S')),'PercFinal']=55

In [192]:
dataset.loc[dataset.MENT2.isin(PartialSuccess),'PercFinal']=55

In [153]:
dataset.head(8)

,ID,ACADYEAR,PERC1,MENT1,PERC2,MENT2,FAC,PROM,Echec,Pass1erSession,Distinction,PercFinal
0,45,2013-2014,NaN,AA,NaN,NaN,FPSE,L2,True,False,False,NaN
1,215,2012-2013,NaN,ADM,63.099998,S,FD,L2,False,False,False,63.099998
2,343,2015-2016,NaN,AA,52.200001,A,FSEG,G2,True,False,False,52.200001
3,356,2015-2016,NaN,ADSTM,59.900002,S,FSEG,L2,False,False,False,59.900002
4,398,2012-2013,65.099998,S,NaN,NaN,FSDC,L2,False,True,False,NaN
5,429,2013-2014,NaN,AA,NaN,A,FD,G1,True,False,False,NaN
6,474,2014-2015,NaN,AA,NaN,NaN,FD,G3,True,False,False,NaN
7,474,2015-2016,62.500000,S,NaN,NaN,FD,G3,False,True,False,NaN


Nous allons remplacer les valeurs manquantes des pourcentage final par un nombre aleatoire symbolisant l'echec

In [122]:
dataset.loc[dataset.Distinction][['PERC1','PERC2','PercFinal','MENT1','MENT2']].MENT1.value_counts()

D         268
AA         74
ADSTM       7
GD          7
A           5
ADM         4
ADTFC       4
ADSTFC      2
ADST        1
Name: MENT1, dtype: int64

In [ ]:
Verifions encore

In [172]:
dataset.loc[np.isnan(dataset.PercFinal)]['MENT2'].value_counts()

AA        515
A         485
NAF        46
ADM        23
ADSTM      19
ADTFC       5
ADST        5
ANAF        3
AR          2
ADSTFC      2
Name: MENT2, dtype: int64

In [173]:
dataset.PercFinal=dataset.PercFinal.apply(lambda x: np.random.choice([x for x in range(40,50)]) if (np.isnan(x)) else x)

In [174]:
dataset.head(8)

,ID,ACADYEAR,PERC1,MENT1,PERC2,MENT2,FAC,PROM,Echec,Pass1erSession,Distinction,PercFinal
0,45,2013-2014,NaN,AA,NaN,NaN,FPSE,L2,True,False,False,43.000000
1,215,2012-2013,NaN,ADM,63.099998,S,FD,L2,False,False,False,63.099998
2,343,2015-2016,NaN,AA,52.200001,A,FSEG,G2,True,False,False,52.200001
3,356,2015-2016,NaN,ADSTM,59.900002,S,FSEG,L2,False,False,False,59.900002
4,398,2012-2013,65.099998,S,NaN,NaN,FSDC,L2,False,True,False,65.099998
5,429,2013-2014,NaN,AA,NaN,A,FD,G1,True,False,False,41.000000
6,474,2014-2015,NaN,AA,NaN,NaN,FD,G3,True,False,False,46.000000
7,474,2015-2016,62.500000,S,NaN,NaN,FD,G3,False,True,False,62.500000


Nous venons de finir le calcul de nos metriques  d'evaluations groupons enfin les metriques pour l'étudiant m

In [175]:
 def f(x):
    A=list(x['Echec'])
    D=list(x['Distinction'])
    S1=list(x['Pass1erSession'])
    Nbre=float(len(x['ACADYEAR']))
    return pd.Series(dict(
     EchecRatio=int(A.count(True))/Nbre,
     DistinctionRatio=int(D.count(True))/Nbre,
     Pass1stSessionRatio=int(S1.count(True))/Nbre,
     CGPA=np.mean(x['PercFinal']),
     NACADYEAR=int(Nbre)
    ))

In [193]:
datasetRatio=dataset.groupby('ID').apply(f).reset_index()

In [194]:
datasetRatio.head(8)

,ID,CGPA,DistinctionRatio,EchecRatio,NACADYEAR,Pass1stSessionRatio
0,45,43.000000,0.0,1.0,1.0,0.0
1,215,63.099998,0.0,0.0,1.0,0.0
2,343,52.200001,0.0,1.0,1.0,0.0
3,356,59.900002,0.0,0.0,1.0,0.0
4,398,65.099998,0.0,0.0,1.0,1.0
5,429,41.000000,0.0,1.0,1.0,0.0
6,474,54.250000,0.0,0.5,2.0,0.5
7,526,65.000000,0.0,0.0,1.0,1.0


In [212]:
datasetRatio.loc[operator.and_(datasetRatio.DistinctionRatio == 1 , datasetRatio.NACADYEAR==3)]

,ID,CGPA,DistinctionRatio,EchecRatio,NACADYEAR,Pass1stSessionRatio
211,4091,72.199999,1.0,0.0,3.0,0.000000
671,5178,71.533333,1.0,0.0,3.0,0.666667
1095,6914,74.900002,1.0,0.0,3.0,1.000000
1928,8498,72.433334,1.0,0.0,3.0,1.000000
2364,9048,71.733332,1.0,0.0,3.0,0.333333
2393,9078,80.766665,1.0,0.0,3.0,1.000000
2830,9527,75.866666,1.0,0.0,3.0,1.000000


In [213]:
dataset.loc[dataset.ID==9078]

,ID,ACADYEAR,PERC1,MENT1,PERC2,MENT2,FAC,PROM,Echec,Pass1erSession,Distinction,PercFinal
5311,9078,2013-2014,80.199997,GD,NaN,NaN,FSTA,G0,False,True,True,80.199997
5312,9078,2014-2015,82.699997,GD,NaN,NaN,FSTA,G1,False,True,True,82.699997
5313,9078,2015-2016,79.400002,D,NaN,NaN,FSTA,G2,False,True,True,79.400002


Le modèle est parfait Nous pouvons maintenant passer à l'analyse des données de sortie Pour nous 2 ensembles d'apprentissage

In [ ]:
dataset.to_csv('DatasetOutV2.csv')
datasetRatio.to_csv('DatasetOutRatioV1.csv')